In [45]:
import pandas as pd 
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
 
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ehmindev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [46]:
df = pd.read_excel("savedrecs.xls")

print(df)

    Publication Type                                            Authors   
0                  C  Deng, J; Dong, W; Socher, R; Li, LJ; Li, K; Li...  \
1                  J  Russakovsky, O; Deng, J; Su, H; Krause, J; Sat...   
2                  C  Szegedy, C; Vanhoucke, V; Ioffe, S; Shlens, J;...   
3                  C            Wang, XL; Girshick, R; Gupta, A; He, KM   
4                  C  Liu, Z; Lin, YT; Cao, Y; Hu, H; Wei, YX; Zhang...   
..               ...                                                ...   
995                J  Aimar, A; Mostafa, H; Calabrese, E; Rios-Navar...   
996                J                 Gong, K; Catana, C; Qi, JY; Li, QZ   
997                J         Hoskere, V; Park, JW; Yoon, H; Spencer, BF   
998                J  Sharif, M; Khan, MA; Zahid, F; Shah, JH; Akram, T   
999                J  Jegham, I; Ben Khalifa, A; Alouani, I; Mahjoub...   

     Book Authors Book Editors Book Group Authors   
0             NaN          NaN               I

In [47]:
df = df[["Article Title","Abstract"]]

In [48]:
docs = df["Abstract"].astype(str).tolist()
docs

['The explosion of image data on the Internet has the potential to foster more sophisticated and robust models and algorithms to index, retrieve, organize and interact with images and multimedia data. But exactly how such data can be harnessed and organized remains a critical problem. We introduce here a new database called ImageNet, a large-scale ontology of images built upon the backbone of the WordNet structure. ImageNet aims to populate the majority of the 80,000 synsets of WordNet with an average of 5001000 clean and full resolution images. This will result in tens of millions of annotated images organized by the semantic hierarchy of WordNet. This paper offers a detailed analysis of ImageNet in its current state: 12 subtrees with 5247 synsets and 3.2 million images in total. We show that ImageNet is much larger in scale and diversity and much more accurate than the current image datasets. Constructing such a large-scale database is a challenging task. We describe the data collect

In [49]:
lemmatizer = WordNetLemmatizer()
a = docs[0].split(" ")
print(a)
print([lemmatizer.lemmatize(word) for word in a])

for i in range(len(docs)):
    docs[i] = "".join([lemmatizer.lemmatize(word) for word in docs[i]])

['The', 'explosion', 'of', 'image', 'data', 'on', 'the', 'Internet', 'has', 'the', 'potential', 'to', 'foster', 'more', 'sophisticated', 'and', 'robust', 'models', 'and', 'algorithms', 'to', 'index,', 'retrieve,', 'organize', 'and', 'interact', 'with', 'images', 'and', 'multimedia', 'data.', 'But', 'exactly', 'how', 'such', 'data', 'can', 'be', 'harnessed', 'and', 'organized', 'remains', 'a', 'critical', 'problem.', 'We', 'introduce', 'here', 'a', 'new', 'database', 'called', 'ImageNet,', 'a', 'large-scale', 'ontology', 'of', 'images', 'built', 'upon', 'the', 'backbone', 'of', 'the', 'WordNet', 'structure.', 'ImageNet', 'aims', 'to', 'populate', 'the', 'majority', 'of', 'the', '80,000', 'synsets', 'of', 'WordNet', 'with', 'an', 'average', 'of', '5001000', 'clean', 'and', 'full', 'resolution', 'images.', 'This', 'will', 'result', 'in', 'tens', 'of', 'millions', 'of', 'annotated', 'images', 'organized', 'by', 'the', 'semantic', 'hierarchy', 'of', 'WordNet.', 'This', 'paper', 'offers', 'a

In [50]:
docs

['The explosion of image data on the Internet has the potential to foster more sophisticated and robust models and algorithms to index, retrieve, organize and interact with images and multimedia data. But exactly how such data can be harnessed and organized remains a critical problem. We introduce here a new database called ImageNet, a large-scale ontology of images built upon the backbone of the WordNet structure. ImageNet aims to populate the majority of the 80,000 synsets of WordNet with an average of 5001000 clean and full resolution images. This will result in tens of millions of annotated images organized by the semantic hierarchy of WordNet. This paper offers a detailed analysis of ImageNet in its current state: 12 subtrees with 5247 synsets and 3.2 million images in total. We show that ImageNet is much larger in scale and diversity and much more accurate than the current image datasets. Constructing such a large-scale database is a challenging task. We describe the data collect

In [51]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english")
ctfidf_model = ClassTfidfTransformer()

In [52]:
model = BERTopic(
  embedding_model=embedding_model,        
  umap_model=umap_model,               
  hdbscan_model=hdbscan_model,              
  vectorizer_model=vectorizer_model, 
  ctfidf_model=ctfidf_model,
  nr_topics=50,
  n_gram_range=(1,2)       
)


In [53]:
topics, probabilities = model.fit_transform(docs)

In [54]:
topic_df = model.get_topic_info()
topic_df

,Topic,Count,Name
0,-1,274,-1_learning_vision_computer_deep
1,0,29,0_tracking_trackers_tracker_object
2,1,43,1_plant_fruit_crop_fruits
3,2,108,2_structural_crack_based_detection
4,3,48,3_face_facial_recognition_expression
5,4,19,4_id_person_pedestrian_vehicle
6,5,39,5_human_action_recognition_activity
7,6,22,6_deep_remote_classification_sensing
8,7,65,7_medical_segmentation_clinical_learning
9,8,104,8_learning_deep_networks_data


In [55]:
get_topic = model.get_topic
get_topic(1)

[('plant', 0.04531534701117882),
 ('fruit', 0.03279738131832232),
 ('crop', 0.02869770865353203),
 ('fruits', 0.025921946195924108),
 ('disease', 0.025796367389158244),
 ('food', 0.02502968685302805),
 ('images', 0.02391900259849522),
 ('species', 0.023682709745086017),
 ('leaf', 0.023423979085956465),
 ('agriculture', 0.022853192344069087)]

In [56]:
zero=[]
number=[]
for i in range(len(topic_df)):
  zero.append('0')
  number.append(i-1)

topic_word_df=pd.DataFrame({
    'topic_num':number,
    'topic':topic_df['Name'],
    'w1':zero,
    'w2':zero,
    'w3':zero,
    'w4':zero,
    'w5':zero,
    'w6':zero,
    'w7':zero,
    'w8':zero,
    'w9':zero,
    'w10':zero
})

# 각 토픽별 10개 단어 정리
col = topic_word_df.columns
for i in range(len(topic_word_df)):
  for j in range(10):
    topic_word_df[col[j+2]][i]=get_topic(i-1)[j][0]

In [57]:
topic_word_df

,topic_num,topic,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10
0,-1,-1_learning_vision_computer_deep,learning,vision,computer,deep,methods,image,object,recognition,based,detection
1,0,0_tracking_trackers_tracker_object,tracking,trackers,tracker,object,target,visual,online,correlation,results,appearance
2,1,1_plant_fruit_crop_fruits,plant,fruit,crop,fruits,disease,food,images,species,leaf,agriculture
3,2,2_structural_crack_based_detection,structural,crack,based,detection,monitoring,inspection,vision,using,data,defect
4,3,3_face_facial_recognition_expression,face,facial,recognition,expression,expressions,faces,databases,computer,age,images
5,4,4_id_person_pedestrian_vehicle,id,person,pedestrian,vehicle,world,open,detection,identification,surveillance,problem
6,5,5_human_action_recognition_activity,human,action,recognition,activity,motion,based,temporal,video,methods,research
7,6,6_deep_remote_classification_sensing,deep,remote,classification,sensing,hyperspectral,data,imagery,land,learning,spatial
8,7,7_medical_segmentation_clinical_learning,medical,segmentation,clinical,learning,deep,image,images,imaging,data,diagnosis
9,8,8_learning_deep_networks_data,learning,deep,networks,data,network,neural,vision,computer,cnn,image


In [58]:
model.get_document_info(docs)

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,The explosion of image data on the Internet ha...,11,11_text_image_language_word,text - image - language - word - captioning - ...,0.702558,False
1,The ImageNet Large Scale Visual Recognition Ch...,13,13_object_detection_detectors_small,object - detection - detectors - small - detec...,0.844850,False
2,Convolutional networks are at the core of most...,8,8_learning_deep_networks_data,learning - deep - networks - data - network - ...,0.781001,False
3,Both convolutional and recurrent operations ar...,14,14_saliency_attention_salient_maps,saliency - attention - salient - maps - detect...,0.444625,False
4,"This paper presents a new vision Transformer, ...",-1,-1_learning_vision_computer_deep,learning - vision - computer - deep - methods ...,0.000000,False
...,...,...,...,...,...,...
995,Convolutional neural networks (CNNs) have beco...,9,9_fpga_cnn_hardware_accelerators,fpga - cnn - hardware - accelerators - acceler...,1.000000,False
996,"Recently, deep neural networks have been widel...",7,7_medical_segmentation_clinical_learning,medical - segmentation - clinical - learning -...,0.855491,False
997,Computer vision techniques for extracting dyna...,2,2_structural_crack_based_detection,structural - crack - based - detection - monit...,1.000000,False
998,Human action recognition from a video sequence...,5,5_human_action_recognition_activity,human - action - recognition - activity - moti...,1.000000,False


In [59]:
model.visualize_barchart()

In [60]:
model.visualize_heatmap()

In [61]:
model.visualize_topics()